<a href="https://colab.research.google.com/github/AlexandreGuerra-prod/Imers-o_Alura/blob/main/agentes_de_ia_versao3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-genai google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 918.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/7

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

In [ ]:
# Cria um serviço de sessão em memória

session_service = InMemorySessionService()

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

# Editada: Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

import asyncio  # Importar para usar asyncio.sleep
from google.api_core import exceptions  # Importar para capturar exceções da API

async def call_agent(agent: Agent, message_text: str) -> str:
    # Número máximo de tentativas
    max_retries = 5
    # Atraso inicial em segundos
    initial_delay = 1

    for attempt in range(max_retries):
        try:
            # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
            session = await session_service.create_session(app_name=agent.name, user_id="user1")
            # Cria um Runner para o agente
            runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
            # Cria o conteúdo da mensagem de entrada
            content = types.Content(role="user", parts=[types.Part(text=message_text)])

            final_response = ""
            # Itera assincronamente pelos eventos retornados durante a execução do agente
            async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
                if event.is_final_response():
                  for part in event.content.parts:
                    if part.text is not None:
                      final_response += part.text
                      final_response += "\n"
            # Se chegou aqui sem erro, retorna a resposta e sai do loop de retry
            return final_response

        except exceptions.ServerError as e:
            # Verifica se é um erro 503 UNAVAILABLE
            if e.code == 503:
                print(f"Erro 503 UNAVAILABLE na tentativa {attempt + 1}/{max_retries}. Tentando novamente...")
                # Calcula o atraso com exponencial backoff
                delay = initial_delay * (2 ** attempt)
                await asyncio.sleep(delay)
            else:
                # Se for outro ServerError, relança a exceção
                raise e
        except Exception as e:
            # Captura outros erros inesperados e relança
            print(f"Erro inesperado na tentativa {attempt + 1}/{max_retries}: {e}")
            raise e

    # Se todas as tentativas falharam, levanta uma exceção
    raise Exception(f"Falha após {max_retries} tentativas de chamar o agente {agent.name}.")

In [ ]:

# Lista todos os modelos disponíveis atualmente

for model in client.models.list():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview

In [ ]:
MODELO_RAPIDO = "gemini-2.0-flash"
MODELO_ROBUSTO = "gemini-1.5-flash"

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

async def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos = await call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################

async def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais.
        Você recebe uma lista de lançamentos recentes do agente buscador e,
        - Para cada um dos lançamentos recebidos, você deve usar a ferramenta
        de busca do Google (google_search) para buscar os pontos mais relevantes
        que poderíamos abordar em um post sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais
        informações sobre cada um dos temas e aprofundar.
        - Depois de terminar a busca, você irá escolher APENAS UM tema dentre todos eles,
        aquele tema com potencial de ser o mais relevante com base nas suas pesquisas.
        - Depois de escolher o tema mais relevante dentre todos eles, você irá retornar
        qual foi o tema escolhido, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"

    # Executa o agente
    plano_do_post = await call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################

async def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model=MODELO_ROBUSTO,
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"

    # Executa o agente
    post_final = await call_agent(redator, entrada_do_agente_redator)
    return post_final


In [ ]:
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")


In [ ]:
print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀\n")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes
if not topico:
    print("\nVocê esqueceu de digitar o tópico!")
else:
    print(f"\nMaravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    post_redigido = await agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(post_redigido))
    print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀

❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes de IA

Maravilha! Vamos então criar o post sobre novidades em agentes de IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais relevantes de agentes de IA no último mês, farei algumas pesquisas no Google para identificar os lançamentos que estão gerando mais notícias e entusiasmo.
> 
> 
> Com base nas notícias e artigos recentes, aqui estão alguns dos lançamentos e desenvolvimentos mais relevantes em agentes de IA no último mês:
> 
> 1.  **Microsoft Build 2025:** A Microsoft está apostando forte na era dos agentes de IA e na construção de uma "open agentic web". No evento Build 2025, a empresa apresentou novos agentes pré-configurados, ferramentas para a criação de agentes personalizados e capacidades multi-agente. O Azure AI Foundry Agent Service também foi anunciado, permitindo que desenvolvedores orquestrem múltiplos agentes especializados para tarefas complexas. A Microsoft Discovery também foi apresentada, uma plataforma para capacitar pesquisadores a usar agentes de IA no processo de descoberta científica.
> 2.  **Grin Intelligent Assistant (Gia) e Charlie:** Empresas de marketing de influência estão lançando agentes de IA para otimizar parcerias com criadores de conteúdo. A Grin lançou o Gia, que identifica potenciais parceiros, personaliza e-mails e sugere valores. A Tano lançou o Charlie, que se comunica diretamente com influenciadores e automatiza a descoberta, abordagem e integração.
> 3.  **StackSpot AI:** A Zup, empresa do grupo Itaú Unibanco, está liderando a transformação no Brasil com a plataforma multiagentes para desenvolvimento de software StackSpot AI. A plataforma combina inteligência artificial generativa e hipercontextualização para transformar o desenvolvimento de software, permitindo uma redução de 50% no lead time.
> 4.  **Agentes de IA para automação de negócios:** A demanda por agentes de IA nos negócios está crescendo, com 82% das empresas planejando integrar agentes de IA nos próximos 1 a 3 anos. Empresas que implantaram agentes de IA relatam melhorias significativas nas operações e aumento na eficiência. O mercado de agentes de IA deve crescer de US$ 5 bilhões em 2024 para mais de US$ 47 bilhões em 2030.
> 5.  **S2, o agente de IA com múltiplas personalidades:** Um novo agente de inteligência artificial criado pela Simular AI, chamado S2, está chamando a atenção por sua estrutura em "múltiplas personalidades". Ele combina diferentes tipos de modelos de IA, cada um responsável por uma função específica, e tem apresentado resultados promissores em testes práticos.
> 
> Esses são apenas alguns dos lançamentos e desenvolvimentos mais relevantes em agentes de IA no último mês. À medida que a tecnologia continua a evoluir, é provável que vejamos ainda mais inovações e aplicações de agentes de IA no futuro.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Ok, com base nos lançamentos recentes de agentes de IA, vou pesquisar mais sobre cada um deles para identificar o tema mais relevante para um post de mídia social.
> 
> 
> Após analisar os resultados da pesquisa, o tema mais relevante para um post de mídia social é: **Microsoft Build 2025 e a Era dos Agentes de IA**.
> 
> **Justificativa:**
> 
> *   **Amplo Interesse:** A Microsoft Build é um evento de grande visibilidade no mundo da tecnologia, e o foco em agentes de IA atrai a atenção de desenvolvedores, empresas e entusiastas da área.
> *   **Inovação:** Os lançamentos da Microsoft, como o Azure AI Foundry Agent Service e o Microsoft Discovery, representam avanços significativos na criação e aplicação de agentes de IA.
> *   **Impacto:** A visão da Microsoft de uma "open agentic web" e a integração de agentes de IA em diversas áreas, como desenvolvimento de software e pesquisa científica, têm o potencial de transformar a forma como interagimos com a tecnologia.
> *   **Atualidade:** O evento Build 2025 é recente, o que garante que o tema esteja em alta e gere engajamento.
> 
> **Plano para o Post:**
> 
> *   **Título:** Microsoft Build 2025: A Era dos Agentes de IA Começou!
> *   **Assuntos a serem abordados:**
> 
>     *   O que são agentes de IA e por que eles são importantes.
>     *   Principais anúncios da Microsoft Build 2025 relacionados a agentes de IA:
>         *   Azure AI Foundry Agent Service: orquestração de múltiplos agentes especializados.
>         *   Microsoft Discovery: uso de agentes de IA na descoberta científica.
>         *   GitHub Copilot evoluído: parceria agentic assíncrona.
>         *   Windows AI Foundry: plataforma unificada para desenvolvimento de IA.
>     *   Exemplos de empresas que já estão utilizando agentes de IA da Microsoft (Fujitsu, NTT DATA, Stanford Health Care).
>     *   A visão da Microsoft de uma "open agentic web" e o futuro da IA.
>     *   O Model Context Protocol (MCP) para garantir segurança e interoperabilidade na web agentic.
> 
> *   **Call to action:**
>     *   Incentivar os seguidores a compartilhar suas opiniões sobre o futuro dos agentes de IA.
>     *   Convidar os desenvolvedores a explorar as novas ferramentas e plataformas da Microsoft.
> *   **Hashtags:** #AgentesIA #InteligenciaArtificial #MicrosoftBuild #AzureAI #OpenAgenticWeb #Inovacao #Tecnologia #FuturoDaIA
> *   **Formato:** O post pode ser um carrossel com imagens e textos concisos, ou um vídeo curto com os principais destaques do evento.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🚀 **Microsoft Build 2025: A Era dos Agentes de IA Começou!** 🚀
> 
> Eita que a Microsoft Build 2025 trouxe MUITA novidade!  Prepare-se para mergulhar na era dos agentes de IA, uma revolução que vai mudar a forma como interagimos com a tecnologia.  
> 
> 🤔 **O que são agentes de IA?**  Imagine assistentes virtuais superpoderosos, capazes de executar tarefas complexas e se comunicar entre si!  Eles estão revolucionando áreas como desenvolvimento de software, pesquisa científica e muito mais.
> 
> A Microsoft apresentou tecnologias incríveis:
> 
> * **Azure AI Foundry Agent Service:** Orquestra vários agentes especializados, trabalhando juntos como uma equipe de super-heróis da tecnologia!
> * **Microsoft Discovery:** Usando agentes de IA para acelerar descobertas científicas, abrindo portas para inovações que antes eram inimagináveis.
> * **GitHub Copilot evoluído:**  Parcerias agentic assíncronas para um desenvolvimento de código ainda mais eficiente e colaborativo.
> * **Windows AI Foundry:** Uma plataforma unificada para criar e implementar agentes de IA, simplificando o processo para desenvolvedores.
> 
> 😱 Empresas gigantes como Fujitsu, NTT DATA e Stanford Health Care já estão utilizando essas ferramentas!
> 
> A Microsoft vislumbra uma "open agentic web", um futuro onde agentes de IA trabalham juntos em perfeita harmonia, tudo com segurança garantida pelo Model Context Protocol (MCP).  
> 
> 🗣️ **E você, o que acha do futuro dos agentes de IA? Compartilhe sua opinião nos comentários!**  E desenvolvedores, preparem-se para explorar essas novas ferramentas incríveis!  Acesse o site da Microsoft para saber mais!  (link na bio)
> 
> #AgentesIA #InteligenciaArtificial #MicrosoftBuild #AzureAI
> 
> 
> 


--------------------------------------------------------------


In [ ]:
# Cria o diretório 'agentes' se ele não existir
!mkdir -p agentes

In [ ]:
%%writefile agentes/redator.py
from google.adk.agents import Agent

def criar_agente_redator(modelo="gemini-1.5-flash"):
    return Agent(
        name="agente_redator",
        model=modelo,
        instruction=\"\"\"
        Você é um redator da Alura. Crie posts para Instagram baseados no plano de conteúdo fornecido.
        Seja criativo, direto, informativo e use hashtags.
        \"\"\",
        description="Cria posts para redes sociais."
    )


Writing agentes/redator.py
